<a href="https://colab.research.google.com/github/Abhi-v-b/LP-5/blob/main/HPC/Assignment%202/Bubble_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-isd5oaeg
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-isd5oaeg
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=f7f5723d2d521f1eaa3c15505c6fc17a6f3303506f651e91874675c1d6bdb85c
  Stored in directory: /tmp/pip-ephem-wheel-cache-190hx4og/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [12]:
%%writefile bubblesort.cu

#include <stdio.h>
#include <time.h>
#include <cuda.h>

#define N 10

__global__ void bubbleSortParallel(int *arr) {
    __shared__ int temp[N];
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = 0; i < N / stride; i++) {
        temp[index] = arr[index];
        if (index + stride < N && arr[index] > arr[index + stride]) {
            temp[index + stride] = arr[index];
            arr[index] = arr[index + stride];
            arr[index + stride] = temp[index];
        }
        if (index + 1 < N && arr[index] > arr[index + 1]) {
            temp[index + 1] = arr[index];
            arr[index] = arr[index + 1];
            arr[index + 1] = temp[index];
        }
    }
}

void bubbleSortSequential(int *arr) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }
}

void printArray(int *arr) {
    for (int i = 0; i < N; i++) {
        printf("%d  ", arr[i]);
    }
    printf("\n");
}

int main() {
    int *arr, *d_arr;
    cudaEvent_t start, stop;
    float time;

    arr = (int *)malloc(N * sizeof(int));
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 100;
    }

    cudaMalloc((void **)&d_arr, N * sizeof(int));
    cudaMemcpy(d_arr, arr, N * sizeof(int), cudaMemcpyHostToDevice);

    printf("Array before sorting:\n");
    printArray(arr);

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    bubbleSortSequential(arr);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time, start, stop);
    printf("\nArray after sequential sorting:\n");
    printArray(arr);
    printf("Sequential execution time: %f ms\n", time);


    cudaEventRecord(start);
    bubbleSortParallel<<<N/32, 32>>>(d_arr);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time, start, stop);
    printf("\nArray after parallel sorting:\n");
    printArray(arr);
    printf("Parallel execution time: %f ms\n", time);

    cudaMemcpy(arr, d_arr, N * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_arr);
    free(arr);

    return 0;
}

Overwriting bubblesort.cu


In [13]:
!nvcc bubblesort.cu -o bubble

In [14]:
!./bubble

Array before sorting:
83  86  77  15  93  35  86  92  49  21  

Array after sequential sorting:
15  21  35  49  77  83  86  86  92  93  
Sequential execution time: 0.002464 ms

Array after parallel sorting:
15  21  35  49  77  83  86  86  92  93  
Parallel execution time: 46.907745 ms
